# Install and configure LDMS-Darshan in the cloud using AWS

Documentation: https://ovis-hpc.readthedocs.io/en/latest/ldms/ldms-streams.html#compile-and-build-with-ldms

Instructions for a single node instance for now.

## AWS Instance

We are using the free-trial eligible instance **t2.micro** to test running Darshan-LDMS in the cloud. 

Select an Ubuntu based instance with default configurations. Click **Lauch instance** and access it through ssh connection:

![static analysis post collecting data](aws1.png)

![static analysis post collecting data](aws2.png)


## LDMS - Install and configure 

In [ ]:
sudo apt-get install autoconf -y 
sudo apt-get install pkg-config -y
sudo apt-get update -y

sudo apt-get install hdf5-tools libhdf5-openmpi-dev openmpi-bin -y
sudo apt-get install python3.10 -y 
sudo apt-get install python-dev-is-python3 -y
sudo apt-get install make -y 
sudo apt-get install bison -y 
sudo apt-get install flex -y
sudo apt-get update -y

git clone https://github.com/ovis-hpc/ovis.git 
cd ovis && mkdir build
./autogen.sh && cd build
../configure --prefix=${HOME}/ovis/build 
make
make install

Create an script to easily setup the LDMS path in the environment - named *set-ldms-env.sh*

The hostname in an instance is defined by its ip address, exp: ip-172-31-35-162

In [ ]:
#!/bin/sh
export LDMS_INSTALL_PATH=/home/ubuntu/ovis/build
export LD_LIBRARY_PATH=$LDMS_INSTALL_PATH/lib/:$LD_LIBRARY_PATH
export LDMSD_PLUGIN_LIBPATH=$LDMS_INSTALL_PATH/lib/ovis-ldms
export ZAP_LIBPATH=$LDMS_INSTALL_PATH/lib/ovis-ldms
export PATH=$LDMS_INSTALL_PATH/sbin:$LDMS_INSTALL_PATH/bin:$PATH
export PYTHONPATH=$LDMS_INSTALL_PATH/lib/python3.10/site-packages

export COMPONENT_ID="1"
export SAMPLE_INTERVAL="1000000"
export SAMPLE_OFFSET="0"
export HOSTNAME="localhost"

## Darshan - Install and configure 

In [ ]:
cd
chmod +x set-ldms-env.sh
source ./set-ldms-env.sh
git clone https://github.com/darshan-hpc/darshan.git
cd darshan
mkdir build
./prepare.sh && cd build/
../configure --with-log-path=${HOME}/darshan/build/logs --prefix=${HOME}/darshan/build/install --with-jobid-env=PBS_JOBID CC=mpicc --enable-ldms-mod --with-ldms=${HOME}/ovis/build
make
make install

The configure stage returns this (unsure if Python module and tools should be recognized):

```
------------------------------------------------------------------------------
   Darshan Utility Version 3.4.4 configured with the following features:
           C compiler                    - mpicc
           Python module and tools       - no
           AUTOPERF MPI  module support  - no
           AUTOPERF APXC module support  - no
------------------------------------------------------------------------------

   Darshan Version 3.4.4

   Features:  Build Darshan runtime libraries - yes
              Build Darshan utility tools     - yes

   Next step: run 'make' to build the library and/or utility tools.
              run 'make install' to install Darshan in directory:
              /home/ubuntu/darshan/build/install

------------------------------------------------------------------------------

```


Create an script to easily setup Darshan in the environment - named *set-ldms-darshan-env.sh*

In [ ]:
#!/bin/sh
export DARSHAN_INSTALL_PATH=/home/ubuntu
export LD_PRELOAD=/home/ubuntu/darshan/build/install/lib/libdarshan.so
export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:$DARSHAN_INSTALL_PATH/lib
export DARSHAN_MOD_ENABLE="DXT_POSIX,DXT_MPIIO"
export DARSHAN_LDMS_STREAM=darshanConnector
export DARSHAN_LDMS_XPRT=sock
export DARSHAN_LDMS_HOST=ubuntu # SET TO THE NODE NAME THE LDMS DAEMON IS RUNNING
export DARSHAN_LDMS_PORT=10444 # SET TO THE PORT WHERE LDMS DAEMON IS RUNNING
export DARSHAN_LDMS_AUTH=none

# enable LDMS data collection. No runtime data collection will occur if this is not exported.
export DARSHAN_LDMS_ENABLE=

# determine which modules we want to publish to ldmsd
export DARSHAN_LDMS_ENABLE_MPIIO=
export DARSHAN_LDMS_ENABLE_POSIX=
export DARSHAN_LDMS_ENABLE_STDIO=
export DARSHAN_LDMS_ENABLE_HDF5=
export DARSHAN_LDMS_ENABLE_ALL=
export DARSHAN_LDMS_VERBOSE=

# darshanConnector
export DARSHAN_LDMS_STREAM=darshanConnector
export DARSHAN_LDMS_XPRT=sock
export DARSHAN_LDMS_HOST=ip-172-31-35-162
export DARSHAN_LDMS_PORT=10444
export DARSHAN_LDMS_AUTH=none

Run to set paths for Darshan:

In [ ]:
chmod +x set-ldms-darshan-env.sh
source ./set-ldms-darshan-env.sh


### Testing LDMS-Darshan:

Create the configuration file for Darshan-LDMS daemon - called *darshan_stream_store.conf*:

In [ ]:
load name=hello_sampler
config name=hello_sampler producer=${HOSTNAME} instance=${HOSTNAME}/hello_sampler stream=darshanConnector component_id=${COMPONENT_ID}
start name=hello_sampler interval=${SAMPLE_INTERVAL} offset=${SAMPLE_OFFSET}

load name=stream_csv_store
config name=stream_csv_store path=./streams/store container=csv stream=darshanConnector rolltype=3 rollover=500000

**[TERMINAL 1]** Run the deamon:

Using sudo to launch an LDMS deamon, it gives permission errors to access the ldmsd.pid file otherwise.

In [ ]:
sudo su
source set-ldms-darshan-env.sh
source set-ldms-env.sh
ldmsd -x sock:10444 -c darshan_stream_store.conf -l /tmp/darshan_stream_store.log -v DEBUG 
cat /tmp/darshan_stream_store.log

Suggestion to always check the log file to see any warning or error debug message.

Check it LDMS is running/connected:

In [ ]:
ldms_ls -p 10444 -x sock -v -v
ps auwx | grep ldmsd | grep -v grep

Output:
```bash
root@ip-172-31-35-162:/home/ubuntu# ldms_ls -p 10444 -x sock -v -v
Hostname    : localhost
IP Address  : 127.0.0.1
Port        : 10444
Transport   : sock
Schema Digest                                                    Schema         Instance                 Flags  Msize  Dsize  Hsize  UID    GID    Perm       Update            Duration          Info    
---------------------------------------------------------------- -------------- ------------------------ ------ ------ ------ ------ ------ ------ ---------- ----------------- ----------------- --------
root@ip-172-31-35-162:/home/ubuntu# ps auwx | grep ldmsd | grep -v grep
ubuntu     55777  0.0  0.5 260092  4952 ?        Ssl  01:24   0:00 ldmsd -x sock:10444 -c darshan_stream_store.conf -l /tmp/darshan_stream_store.log -v DEBUG
```

No queries set:
```bash
ubuntu@ip-172-31-35-162:/tmp/darshan-ldms-test$ ldms_ls -h localhost -x sock -p 10444 -v 
Schema         Instance                 Flags  Msize  Dsize  Hsize  UID    GID    Perm       Update            Duration          Info    
-------------- ------------------------ ------ ------ ------ ------ ------ ------ ---------- ----------------- ----------------- --------
```

**[TERMINAL 2]** In another terminal, run python code:

In [ ]:
source set-ldms-darshan-env.sh
source set-ldms-env.sh
export PROG=mpi-io-test
export DARSHAN_TMP=/tmp/darshan-ldms-test
export DARSHAN_TESTDIR=/home/ubuntu/darshan/darshan-test/regression
export DARSHAN_LOGFILE=$DARSHAN_TMP/${PROG}.darshan

mkdir -p $DARSHAN_TMP
cd $DARSHAN_TESTDIR
mpicc $DARSHAN_TESTDIR/test-cases/src/${PROG}.c -o $DARSHAN_TMP/${PROG}
cd $DARSHAN_TMP
sudo ./${PROG} -f $DARSHAN_TMP/${PROG}.tmp.dat

Application output:
```bash
sudo ./${PROG} -f $DARSHAN_TMP/${PROG}.tmp.dat
# Using mpi-io calls.
nr_procs = 1, nr_iter = 1, blk_sz = 16777216, coll = 0
# total_size = 16777216
# Write: min_t = 0.006564, max_t = 0.006564, mean_t = 0.006564, var_t = 0.000000
# Read:  min_t = 0.010569, max_t = 0.010569, mean_t = 0.010569, var_t = 0.000000
Write bandwidth = 2555.935759 Mbytes/sec
Read bandwidth = 1587.377742 Mbytes/sec
```

Check results collected by the LDMS stream: - **RETURNING EMPTY FOR NOW!!!**
```bash
cat /tmp/darshan_stream_store.log
```

# Using multiple nodes in the cloud [to-do]

```bash
sudo apt install slurm-client
```